```
This script can be used for any purpose without limitation subject to the
conditions at http://www.ccdc.cam.ac.uk/Community/Pages/Licences/v2.aspx

This permission notice and the following statement of attribution must be
included in all copies or substantial portions of this script.

2022-06-01: Made available by the Cambridge Crystallographic Data Centre.

```

# Ligand Preparation for Covalent Docking

This notebook illustrates the preparation of ligands for covalent docking with GOLD using a small set of acrylamide ligands to be docked into COVID-19 Main Protease (MPro). 

**Important!** Currently, covalent docking cannot be fully configured using the API. We thus suggested that the docking is set up _via_ Hermes. Once a working configuration has been created, it can be used _via_ the API or the GOLD HPC tools.

#### Docking basics

For optimal performance, GOLD requires good-quality 3D ligand structures as input (note that it only requires a single conformer as it performs flexible docking based on the input structure).

In these documents, a '1D' structure means one where no meaningful 2D or 3D coordinates are present, whereas a '3D' structure is one where a realistic conformation is present. A '2D' structure would be one with a useful 2D depiction, but we are not really concerned with that here.

Note that it is assumed that the input structures are all in the desired charge and tautomeric states. No protonation/deprotonation or tautomer standardization/enumeration is done here.


#### Covalent docking specifics

Preparing ligands for covalent docking is more complicated that normal due to the way covalency is handled in GOLD. Familiarity with Section 5.6 of the GOLD User Guide, 'Setting Up Covalently Bound Ligands' is assumed in what follows.

In Section 5.6.1, 'Method Used for Docking Covalently Bound Ligands', it is stated: 

> The program assumes that there is just one atom linking the ligand to the protein (e.g. the O in a serine residue). Both protein and ligand files are set up with the link atom included (so, if the serine O is the link atom, it will appear in both the protein and ligand input files). Ideally the link atom, in both the ligand and the protein, will have a free valence available through which the link can be made."
> 
> Inside the GOLD least-squares fitting routine, the link atom in the ligand will be forced to fit
onto the link atom in the protein.

In this example, a cysteine will be used as the nucleophilic residue, so the linker atom will be a sulphur instead of an oxygen. The same principles apply, however.

As the link atom must be included in the ligands input to GOLD, the input structures must first be transformed into the adducted form. These structure are artificial, of course, as the protein is obviously not present and the linker atom must have an unfilled valence.

Now, the CCDC API does not currently provide facilities for chemical transformations, so we again recommend the use of RDKit (chemical reaction handling is described [here](https://www.rdkit.org/docs/GettingStartedInPython.html#chemical-reactions)).

Note that 'chemical reaction' is used very loosley here: the transformations specified do not need to correspond to known chemical reactions. In addition, multiple reaction steps may compressed into a single step for conveniance. For example, the nucleophilic addition and keto-enol rearrangement step implicit in the reaction of a vinyl ketone moeity (as in an acrylamide) with a nucleophile (such as a protein thiol) can be compressed into one 'reaction' as is done below.

All the molecules in this test set contain an unsubstituted vinyl ketone as part if the acrylamide (_i.e._ it is substituted only on the nitrogen). This is important as the reaction with the thiol will not introduce any new stereocentres into the molecule. If one or both carbons of the vinyl group are asymmetrically substituted then the adduct will contain one or two new chiral centres. This can certainly be handled withing this general framework but inevitably introduces some complexity. For simplicity we thus focus here on the unsubstituted case; a notebook discussing the effects of substitution of the vinyl group is available on request.

Although it is not an issue for the current example (due to the focus on the unsubtituted case), an important point to note is that the RDKit reaction SMARTS can be written with varying levels of strictness with regards to hydrogens. They can be written with exact hydrogen counts on all relevant atoms or written partially or wholly without hydrogens. In the latter case, RDKit must be relied upon to create valid molecules. It is generally good at this and the exceptions can be ususally be fixed using the [Chem.SanitizeMol](https://www.rdkit.org/docs/source/rdkit.Chem.rdmolops.html?highlight=sanitizemol#rdkit.Chem.rdmolops.SanitizeMol) method. This method imposes little overhead and we use it routinely here to avoid any potential problems. 

A closely related issue is how tightly the reacting substructures are specified. Using SMARTs primitives for _e.g._ valence and total connectivities (as well as hydrogen counts) the reaction can be restricted to only well defined cases. This is generally desirable as it means the products will be well understood; loosely-defined reactions can lead to unexpected or multiple products, issues with stereocentres _etc_. The disadvantage is that more heterogeneous input sets might need several reactions in order to cover all cases. In general, I would recomment defining the reaction as tightly as necessary given the likely inputs. This can mean doing some preliminary exploratory work to determine the diversity present in the input set, which would probably consist of some combination of visualization inspection and substructure searches. Some brief examples of such validation are included below. 

Another point is that the linker atom must have an 'unphysical' open valence, but that a chemically-valid molecule is required for the conformer generator. Our approach here is to use a 'chemical tag' on the thiol reactant whcih is easy to identify and remove in a final step. Astatine is used here as it is conveniently momovalent and accepted by the conformer generator but is highly unlikely to be present in the input mulecules.



#### Notes on the docking system

A functional docking system is provided here for demonstration purposes.

The protein target is derived from [5REN](https://www.ebi.ac.uk/pdbe/entry/pdb/5ren), which is SARS-CoV-2 (_i.e._ COVID-19) Main Protease with a fragment inhibitor covalently bound to the catalysic cysteine (Cys145). The file `target\protein.mol2` was prepared by importing the PDBe structure into Hermes and performing basic protein-preparation steps using the GOLD Wizard (see the GOLD and Hermes User Guides and asociated CCDC tutorial materials for details).

Note that the preparation of this demo system has not been performed thoroughly, _e.g._ Asn and Gln orientations and His tautomers and orientatations have not been investigated in any detail. This could be important here as there are His residues in and around the active site. Thus, while the provided system is suitable for demo purposes, it should not be considered suitable for production work.

The bound inhibitor was extracted and saved as `target\ligand.mol2` for the purposes of cavity-definition only: it takes no other role in the docking. The ligand extraction leaves the bare linker S atom on the protein (Cys145.SG), as is required for covalent docking. 

When covalent docking multiple ligands with a common warhead, it is best to use a substructure file to define the linker: see Section 5.6.3, 'Setting Up Substructure-Based Covalent Links' in the GOLD User Guide for details. The substructure file must be in in `MOL2` format and contain appropriate Sybyl atom types. The file `substructure.mol2` provided with this example was generated from the simple example that is used below. This was exported from the notebook as a `MOL` file and loaded into Hermes, which causes Sybyl atom types to be assigned. These were visually inspected and found appropriate: the required substructure was then created by manually deleting the methyl substituents on the acrylamide N and exporting the the in the default format (_i.e._ `MOL2`). The atom index of the linker S atom was then noted and used in the setup of the docking in Hermes, as was the index of the common S atom in the protein (_i.e._ Cys145.SG).

The `.conf` files provided have settings suitable for a (medium throughput) virtual screen: again, it should not be used for production work without further efforts to establish it's suitability.

Please remember that the ligands used in the example are just for illustrative purposes and are not claimed to be active!

#### Postprocessing

It can be useful to export docking results as complete protein-ligand complexes, if, for example, further computations are to be performed. Howeever, the normal ways of making complexes do not quite work as expected for GOLD covlent dockings. This is explored further in the pair of companion notebooks [Covalent_Complexes-atom](02a_Covalent_Complexes-atom.ipynb) and [Covalent_Complexes-substructure](02b_Covalent_Complexes-substructure.ipynb).

In [1]:
from platform import platform
import sys
import os
from pathlib import Path
import logging
import time
import subprocess
import pandas as pd
import csv
import shutil

In [2]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
from rdkit.Chem.Draw import IPythonConsole

PandasTools.RenderImagesInAllDataFrames()
IPythonConsole.ipython_useSVG = True

ImportError: DLL load failed while importing rdmolfiles: The specified procedure could not be found.

In [ ]:
from IPython.display import HTML, SVG

In [ ]:
import ccdc

from ccdc.io import EntryReader, EntryWriter, MoleculeReader, MoleculeWriter
from ccdc.entry import Entry
from ccdc.molecule import Molecule
from ccdc.diagram import DiagramGenerator
from ccdc.conformer import ConformerGenerator
from ccdc.docking import Docker

#### Config

Dir from which files will read and written...

In [ ]:
working_dir = Path('.')

File of input structures in CSV format, including SMILES and name column and assumed to be in current directory...

In [ ]:
input_csv =  working_dir / 'input.csv'  # Required input file

smiles_col, name_col = 'smiles', 'name'  # Required columns

Output molfile for the ligand preparation process, _i.e._ the input file for GOLD...

_N.B._ The file extension will determine the format written.

In [ ]:
output_format = 'sdf'

In [ ]:
output_molfile = working_dir / f'input.{output_format}'

Pandas display options...

In [ ]:
pd.options.display.max_colwidth = 100

Utility to improve renedering of DataFrames...

In [ ]:
show_df = lambda df: df.style.set_properties(**{'text-align': 'left'})

#### Initialization

In [ ]:
# Set up and configure a logger...

logger = create_logger()

In [ ]:
# Set up and configure a CCDC Diagram Generator...

diagram_generator = DiagramGenerator()

diagram_generator.settings.return_type = 'SVG'
diagram_generator.settings.explicit_polar_hydrogens = False
diagram_generator.settings.shrink_symbols = False

mol2html = lambda mol: HTML(diagram_generator.image(mol))  # Utility to display an API depiction

In [ ]:
# Check that all required input files exist...

for file in [input_csv]: assert file.exists(), f"Error! Required file '{file}' not found."

### Conversion of acrylamides to thiol adducts

Here we illustrate the conversion of an acrylamide to an adduct.

The reaction used is defined below. Note that hydrogen counts are fully specified in both the reactant and products.

A version without these counts is provided below in case you would to experiment with it.

In [ ]:
reaction = AllChem.ReactionFromSmarts('[O:1]=[C:2]-[CH:3]=[CH2:4].[Sv2H:5]>>[O:1]=[C:2]-[CH2:3]-[CH2:4]-[SH0:5]')

reaction  # RDKit can depict reactions

In [ ]:
# # Version without hydrogen counts...

# reaction = AllChem.ReactionFromSmarts('[O:1]=[C:2]-[C:3]=[C:4].[Sv2:5]>>[O:1]=[C:2]-[C:3]-[C:4]-[S:5]')

Get a simple example acrylamide...

In [ ]:
mol = Chem.MolFromSmiles('CN(C)C(=O)C=C')  # Dimethyl acrylamide

mol

Instantiate the tagged thiol nucleophile...

**Important!** Please note that `At` is used as a tag as it has a suitable valence and it is highly unlikely to ever occur in an input molecule: it has absolutely no chemical significance and is discarded later in the procedure.

In [ ]:
tag = '[At]'

tagged_nucleophile = Chem.MolFromSmiles('S' + tag)

Run the reaction...

In [ ]:
products = reaction.RunReactants((mol, tagged_nucleophile))

There is only one product, which is our adduct...

In [ ]:
assert (len(products) == 1) and (len(products[0]) == 1), "Error! No product produced by reaction."

new_mol = products[0][0]  # Extract adduct

Chem.SanitizeMol(new_mol)  # Just in case

print(Chem.MolToSmiles(new_mol))

new_mol

We can run same procedure on the molecules in the dataframe to inspect the results of the transformation on real molecules. 

#### Load and validate input structures

First we load, inspect and validate the input acrylamide structures...

In [ ]:
mols_df = pd.read_csv(input_csv)

assert all(col in mols_df.columns for col in [smiles_col, name_col]), f"Error! Required column not present in '{input_csv}'."  # Check required cols are present

PandasTools.AddMoleculeColumnToFrame(mols_df, smilesCol='smiles', molCol='mol')

mols_df.shape

In [ ]:
mols_df.head()

Check all mols indeed contain an unsubstituted vinyl ketone as part of the acrylamide warhead...

In [ ]:
warhead_query = Chem.MolFromSmarts('[Nv3X3]C(=O)[CH]=[CH2]')

warhead_query

In [ ]:
assert all(mols_df['mol'] >= warhead_query)

Check no input molecules already contain the tag, as it is to be removed later to provide the free valence...

In [ ]:
assert not any(mols_df['mol'] >= Chem.MolFromSmiles(tag))

#### Run reaction on input structure

Define a simple function to run the reaction on an input molecule...

In [ ]:
def run_reaction(mol):
    
    products = reaction.RunReactants((mol, tagged_nucleophile))
    
    assert (len(products) == 1) and (len(products[0]) == 1), "Error! No product produced by reaction."

    new_mol = products[0][0]  # Extract adduct

    Chem.SanitizeMol(new_mol)  # Sanitize

    return new_mol

Run the reaction on the input molecules and add them to the dataframe for easy inpection...

In [ ]:
mols_df = mols_df.assign(new_mol = lambda df: df['mol'].apply(run_reaction))

mols_df.shape

In [ ]:
show_df(mols_df.head())

Check that all the processed mols contain the expected tagged adduct...

In [ ]:
adduct_query = Chem.MolFromSmarts(tag + '-[SX2]-[CH2]-[CH2]-C(=O)')

adduct_query

In [ ]:
assert all(mols_df['new_mol'] >= adduct_query)

### The 1D to 3D conversion

Here we illustrate the conversion of an adduct to a 3D structure suitable for docking.

In [ ]:
# Initialise a conformer generator: recall that only a single conformer is required, as GOLD performs flexible docking...

conformer_generator = ConformerGenerator()

conformer_generator.settings.max_conformers = 1 

We will continue using our simple example...

In [ ]:
new_mol

Convert 1D RDKit mol to 3D API mol _via_ a string representation (we use a molblock here, but could also use SMILES)...
...

Note that hydrogens are added automatically by the conformer generator and that we take a copy of the resulting molecule as we need to edit this and this cannot be done to conformers by default.

In [ ]:
molblock = Chem.MolToMolBlock(new_mol)  # String representation from RDKit molecule

mol = Molecule.from_string(molblock, format='mol')  # 1D API molecule

mol = conformer_generator.generate(mol)[0].molecule.copy()  # Editable 3D molecule

Note that the linker atom is tagged...

In [ ]:
mol2html(mol)  # 2D depiction of 3D mol

To remove the tag and leave a 'bare' linker atom, we will use a [SubstructureSearch](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/substructure_searching.html#setting-up-and-running-a-substructure-search) object...

In [ ]:
tag_searcher = ccdc.search.SubstructureSearch()

_ = tag_searcher.add_substructure(ccdc.search.SMARTSSubstructure(tag))

In [ ]:
# As the tag was present in the original mol (see above), only one, single-atom substructure can be present in the hit

tag_matches = tag_searcher.search(mol) 

assert len(tag_matches) == 1 and len(tag_matches[0].match_atoms()) == 1  # Should be a single match consisting of a single atom

tag_atom = tag_matches[0].match_atoms()[0]  # Tag atom

mol.remove_atom(tag_atom)  # Remove tag atom

Note that the linker atom is now 'bare', _i.e._ has an unfilled valence...

In [ ]:
mol2html(mol)  # 2D depiction

This molecule is now ready for covalent docking, so write out a molfile for input into GOLD...

In [ ]:
with EntryWriter('test.mol') as writer:
    
    writer.write(mol)

Inspect molecule in Hermes...

In [ ]:
status = subprocess.Popen([hermes_exe.as_posix(), 'test.mol'])  # Start Hermes in background

In [ ]:
# os.unlink('test.mol')  # Tidy up

If, in Hermes, you label the atoms with their ID numbers (`Right-Click > Labels > Label by Atom File Index`) you will see that the S linker atom is labelled 5. This is because of the way the molecule is prepared (see the numbering of the reaction SMARTS above) and will be the same for all molecules prepared this way. This fact can be used to drive the covalent docking: see the input file `gold_atom.conf` for an example of how to do this (see also the companion notebook [Covalent_Complexes‑atom](02a_Covalent_Complexes-atom.ipynb)).

The docking may also be driven by using a substructure: see the input file `gold_substructure.conf` for an example of how to do this (see also the companion notebook [Covalent_Complexes‑substructure](02b_Covalent_Complexes-substructure.ipynb)). As noted above, this substructure-based method is usually to be preferred as it is somewhat more general: however, if you wish to use the atom-based method then thre is no issue with it if the molecules are properly prepared.

### The complete process

Here we run through the whole process in a way that could easily be extracted from the notebook and implemented as a standalone script. Such a script could then also be modified to process other systems.

We do not use Pandas here as it is not necessary and would introduce an unecessary dependency for the script. Note that error checking is fairly lightweight and a script could (and should) easily be made more rubust.

Instantiate the tagged nucleophile...

In [ ]:
tag = '[At]'

tagged_nucleophile = Chem.MolFromSmiles('S' + tag)  # Thiol only

Initialise a substructure searcher for removing the tag...

In [ ]:
tag_searcher = ccdc.search.SubstructureSearch()

_ = tag_searcher.add_substructure(ccdc.search.SMARTSSubstructure(tag))

Initialise the conformer generator...

In [ ]:
conformer_generator = ConformerGenerator()

conformer_generator.settings.max_conformers = 1  # GOLD only requires a single conformer

Read the input CSV file, process the SMILES and write the processed mols to an output SDF file....

In [ ]:
MAX_MOLS = 10  # For demo, limit number to process for speed

In [ ]:
logger.info(f"Starting to process input file '{input_csv}'...")

with input_csv.open() as file:  # Open input CSV file for reading

    reader = csv.DictReader(file)  # Initialise CSV reader

    assert all(col in reader.fieldnames for col in [smiles_col, name_col])  # Check required columns are present

    with EntryWriter(str(output_molfile)) as writer:  # Open output molfile file for writing

        for mol_n, record in enumerate(reader, 1):

            smiles, name = record[smiles_col], record[name_col]  # Extract required data from record

            # Convert input SMILES to 2D RDKit mol...

            input_mol = Chem.MolFromSmiles(smiles)

            # Tranform input molecule to adduct with linker tagged...

            products = reaction.RunReactants((input_mol, tagged_nucleophile))

            if not (len(products) == 1) and (len(products[0]) == 1):

                logger.warning(f"Error! No product produced by reaction for mol '{name}'.")

                continue

            new_mol = products[0][0]  # Extract adduct

            Chem.SanitizeMol(new_mol)  # Precautionary

            # Convert 2D RDKit mol to 3D API mol...
 
            molblock = Chem.MolToMolBlock(new_mol)  # String representation from RDKit molecule

            mol = Molecule.from_string(molblock, format='mol')  # 2D API molecule

            mol = conformer_generator.generate(mol)[0].molecule.copy()  # Editable 3D molecule

            mol.identifier = name  # The identifier gets written as the name in the the mol block header

            # Remove the tag to leave a 'bare' linker, as required by GOLD...

            tag_matches = tag_searcher.search(mol)  # Should be a single match consisting of a single atom

            mol.remove_atom(tag_matches[0].match_atoms()[0])  # Remove tag atom

            # Write to file, copying over any input properties...

            writer.write(Entry.from_molecule(mol, index=mol_n, **record))

            logger.info(f"{mol_n:3d}) completed mol '{name}'.")

            if mol_n == MAX_MOLS: break  # For demo, limit number to process for speed

logger.info(f"Finished.")

A docking may now be run with one of the supplied GOLD configuration files and the ligands prepared here.

This can be done in Hermes: `GOLD > Setup and Run a Docking > Load Existing > gold.conf`.

Or the GOLD executable could be used:

1) Run docking using atom-based method (then see companion notebook [Covalent_Complexes‑atom](02a_Covalent_Complexes-atom.ipynb))...

In [ ]:
%%time

conf_file = 'gold_atom.conf'
settings = Docker.Settings.from_file(conf_file)

docker = Docker(settings=settings)

results = docker.dock()

# Examine the results in the GUI

run_hermes(conf_file)



2) Run docking using the (generally preferred) substructure-based method (then see companion notebook [Covalent_Complexes‑substructure](02b_Covalent_Complexes-substructure.ipynb))...

In [ ]:
%%time

conf_file = 'gold_substructure.conf'

settings = Docker.Settings.from_file(conf_file)

docker = Docker(settings=settings)

results = docker.dock()

# Examine the results in the GUI

run_hermes(conf_file)